<a href="https://colab.research.google.com/github/hughesymendes/TIC-27491137/blob/master/Detrending_Bugged_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KeplerGO/lightkurve.git

Cloning into 'lightkurve'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 10978 (delta 1), reused 2 (delta 0), pack-reused 10954
Receiving objects: 100% (10978/10978), 119.58 MiB | 36.87 MiB/s, done.
Resolving deltas: 100% (8184/8184), done.


In [2]:
cd /content/lightkurve/

/content/lightkurve


In [3]:
!pwd

/content/lightkurve


In [4]:
!pip install lightkurve -e .

Obtaining file:///content/lightkurve
     |████████████████████████████████| 2.1MB 4.6MB/s 
     |████████████████████████████████| 6.7MB 27.7MB/s 
     |████████████████████████████████| 25.9MB 84kB/s 
     |████████████████████████████████| 6.5MB 50.0MB/s 
     |████████████████████████████████| 256kB 53.8MB/s 
     |████████████████████████████████| 2.7MB 49.1MB/s 
  Created wheel for astroquery: filename=astroquery-0.4.1-cp36-none-any.whl size=3831873 sha256=35558906121c3ac1dca4c7d7aa11ea760cdbaba97aa1a854a2f6f0cc0b118030
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
  Created wheel for oktopus: filename=oktopus-0.1.2-cp36-none-any.whl size=12779 sha256=953aad6dadd0ea8ec26f3870f3368db8bf9c7e9723d61eb17211b9f5ce32abb3
  Stored in directory: /root/.cache/pip/wheels/9b/90/81/098fc66ee56166d63c9a8fc0a9672ae7b3423396a588ec952a
  Created wheel for fbpca: filename=fbpca-1.0-cp36-none-any.whl size=11376 sha256=3f89c5b91dc449bdbce

In [5]:
!pip install exoplanet

!pip install pymc3==3.9.2

!pip install corner

     |████████████████████████████████| 1.7MB 5.2MB/s 
     |████████████████████████████████| 1.9MB 5.4MB/s 
     |████████████████████████████████| 1.5MB 45.8MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 4.3MB 55.9MB/s 
     |████████████████████████████████| 296kB 55.2MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=edab636f575c681ef617d826861e3a2fdda057694b84c376daf57602c96d62bd
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
  Found existing installation: pymc3 3.7
    Uninstalling pymc3-3.7:
      Successfully uninstalled pymc3-3.7


In [6]:
import lightkurve as lk
lk.__version__

'2.0.dev'

In [7]:
from lightkurve import search_targetpixelfile
from lightkurve import TessTargetPixelFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
%matplotlib inline

flags = lk.utils.TessQualityFlags
q = flags.CoarsePoint | flags.Desat | flags.Argabrightening

tpfs = lk.search_targetpixelfile("TIC 27491137").download_all(quality_bitmask=q)


clcs = []
for tpf in tpfs:
    diff = np.diff(tpf.time.jd)
    av = np.median(np.diff(tpf.time.jd))
    
    breaks = np.where(np.append(False, diff > av * 5))[0]
    aper = tpf.create_threshold_mask()

    aper = tpf.create_threshold_mask()

    
    lc = tpf.to_lightcurve()

    # Scattered light
    bkg = tpf.get_bkg_lightcurve(tpf.pipeline_mask)
    bkg[~np.isfinite(lc.flux_err)].flux *= 0
    lc += bkg
    lc.flux_err[~np.isfinite(lc.flux_err)] = np.nanmedian(lc.flux_err)

    bkg = tpf.flux_bkg
    bkg[~np.isfinite(bkg)] = 0
    pixels = (tpf.flux + bkg)[:, ~aper]
    dm1 = lk.DesignMatrix(pixels, name='pixels').pca(3).split(list(breaks)).append_constant()
    
    # centroids
    t = (lc.cadenceno - np.median(lc.cadenceno))/(len(lc.time))
    ts = np.vstack([((tpf.time - t[0])/(t[-1] - t[0]) - 0.5) * np.in1d(tpf.time, t) for t in np.array_split(tpf.time, [np.where(np.diff(tpf.time) > 0.5)[0][0] + 1])]).T
    p1 = np.vstack([np.nan_to_num(tpf.pos_corr1) * np.in1d(tpf.time, t)  for t in np.array_split(tpf.time, [np.where(np.diff(tpf.time) > 0.5)[0][0] + 1])]).T
    p2 = np.vstack([np.nan_to_num(tpf.pos_corr2) * np.in1d(tpf.time, t)  for t in np.array_split(tpf.time, [np.where(np.diff(tpf.time) > 0.5)[0][0] + 1])]).T
    pdm = lk.DesignMatrix(np.hstack([ts, ts**2, p1, p2, p1*p2, p1**2*p2, p1*p2**2, p1**2*p2**2]), name = 'poscorrs')
    pdm.prior_sigma = np.ones(pdm.shape[1]) * 0.01

    
    spline = lk.designmatrix.create_spline_matrix(lc.time, n_knots=80)

    dm = lk.DesignMatrixCollection([dm1, pdm, spline])

    # First iteration
    r = lk.RegressionCorrector(lc)
    clean_lc = r.correct(dm, sigma=1e10)

    # YOU SHOULD REPLACE THIS HACKY MASK WITH A BETTER ONE.
    a = (clean_lc).bin(10).flatten(81)
    _, m = a.remove_outliers(2, return_mask=True)
    m = (convolve(m, Box1DKernel(10)) > 0.5)
    mask = np.interp(clean_lc.time, a.time, m) > 0
    mask = convolve(mask, Box1DKernel(5)) > 0

    # second iteration
    r = lk.RegressionCorrector(lc)
    _ = r.correct(dm, sigma=1e10)
    corr = (r.diagnostic_lightcurves['poscorrs'] + r.diagnostic_lightcurves['pixels'])
    clean_lc = (lc - corr).normalize()

    clean_lc = lk.LightCurve(np.asarray(clean_lc.time, np.float64),
                         np.asarray(clean_lc.flux, np.float64),
                         np.asarray(clean_lc.flux_err, np.float64))

    fig, ax = plt.subplots(figsize=(30, 4))
    (clean_lc).bin(5, method='median').errorbar(ls='-', ax=ax, c='k')
    
    clcs.append(clean_lc.normalize())

/content/lightkurve/lightkurve/correctors/designmatrix.py:287: LightkurveWarning: The design matrix has low rank (4) compared to the number of columns (94), which suggests that the matrix contains duplicate or correlated columns. This may prevent the regression from succeeding. Consider reducing the dimensionality by calling the `pca()` method.
  LightkurveWarning)


KeyboardInterrupt: ignored

In [ ]:
########## No need to run past here, just trying to debug whats going on with specific parts of the code ##########

In [ ]:
np.median(np.diff(tpf.time))

<TimeDelta object: scale='tdb' format='jd' value=0.0013888663786474353>

In [ ]:
 np.diff(tpf.time)

array([<TimeDelta object: scale='tdb' format='jd' value=0.0013888447247154545>,
       <TimeDelta object: scale='tdb' format='jd' value=0.0013888449570913508>,
       <TimeDelta object: scale='tdb' format='jd' value=0.0013888447247154545>,
       ...,
       <TimeDelta object: scale='tdb' format='jd' value=0.0013888861681152775>,
       <TimeDelta object: scale='tdb' format='jd' value=0.001388886168570025>,
       <TimeDelta object: scale='tdb' format='jd' value=0.001388886168570025>],
      dtype=object)

In [ ]:
np.median(np.diff(tpf.time.jd))

0.0013888664543628693